# Utilisation de Tensorflow pour classifier des images

## Utilisation d'un modèle déjà tout fait

In [ ]:
#!pip install tensorflow numpy pillow

In [1]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
import numpy as np
from PIL import Image
import requests
from io import BytesIO

model = MobileNetV2(weights='imagenet')

def preprocess(image_url):
  response = requests.get(image_url)
  img = Image.open(BytesIO(response.content))
  img = img.resize((224, 224))
  img = np.expand_dims(img, axis=0)
  img = preprocess_input(img)
  return img

def classify(img):
  predictions = model.predict(img)
  decoded_predictions = decode_predictions(predictions, top=3)[0]
  for label in decoded_predictions:
    print(f"{label[1]}: {label[2]:.2f}")

2023-10-24 14:25:58.179277: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Exception: URL fetch failure on https://storage.googleapis.com/tensorflow/keras-applications/mobilenet_v2/mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_224.h5: None -- [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:997)

### Tests

In [ ]:
print("Bonsoir !")

Bonsoir !


## Création du modèle "à la main"

In [ ]:
import tensorflow_datasets as tfds

dataset, info = tfds.load("cifar10", with_info=True, as_supervised=True)
train_dataset, test_dataset = dataset["train"], dataset["test"]

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cifar10/3.0.2.incompleteOD14VA/cifar10-train.tfrecord*...:   0%|          …

Generating test examples...:   0%|          | 0/10000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cifar10/3.0.2.incompleteOD14VA/cifar10-test.tfrecord*...:   0%|          |…

Dataset cifar10 downloaded and prepared to /root/tensorflow_datasets/cifar10/3.0.2. Subsequent calls will reuse this data.


In [ ]:
def preprocess_image(image, label):
    image = tf.image.resize(image, (128, 128))
    image = tf.image.convert_image_dtype(image, tf.float32)
    return image, label

train_dataset = train_dataset.map(preprocess_image)
test_dataset = test_dataset.map(preprocess_image)


In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([
    layers.Conv2D(filters=16, kernel_size=3, activation='relu', padding='same',
                  input_shape=[128, 128, 3]),
    layers.MaxPool2D(),

    layers.Conv2D(filters=32, kernel_size=3, activation='relu', padding='same'),
    layers.MaxPool2D(),

    layers.Flatten(),
    layers.Dense(6, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(1, activation='sigmoid'),
])

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

batch_size = 100 # On ne met pas à jour les poids à chaque donnée d'entraînement, on fait une moyenne sur un batch
num_epochs = 1 # Le modèle va passer 10 fois sur le dataset en entier
train_steps = len(train_dataset) // batch_size
test_steps = len(test_dataset) // batch_size

model.fit(train_dataset.batch(batch_size), epochs=num_epochs, steps_per_epoch=train_steps)


 13/500 [..............................] - ETA: 44:38 - loss: -236607552.0000 - accuracy: 0.0854

KeyboardInterrupt: ignored